In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, random_split, DataLoader
from torch import nn
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torchmetrics.classification import MultilabelF1Score
from sklearn.model_selection import KFold, StratifiedKFold
import warnings
import copy

warnings.filterwarnings("ignore")
plt.style.use('ggplot')

MAIN_DIR = "/workspace/data"

In [2]:
class Config:
    """Configuration class for paths and hyperparameters."""
    train_sequences_path = MAIN_DIR + "/Train/train_sequences.fasta"
    train_labels_path = MAIN_DIR + "/Train/train_terms.tsv"
    test_sequences_path = MAIN_DIR + "/Test/testsuperset.fasta"
    
    num_labels = 500
    n_epochs = 20
    batch_size = 128
    lr = 0.01
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


embeds_map = {
    "T5": "t5embeds",
    "ProtBERT": "protbert-embeddings-for-cafa5",
    "EMS2": "cafa-5-ems-2-embeddings-numpy"
}

embeds_dim = {
    "T5": 1024,
    "ProtBERT": 1024,
    "EMS2": 1280
}

In [3]:
class ProteinSequenceDataset(Dataset):
    """Dataset class for protein sequences with embeddings."""
    
    def __init__(self, datatype, embeddings_source):
        super(ProteinSequenceDataset, self).__init__()
        self.datatype = datatype
        
        if embeddings_source in ["ProtBERT", "EMS2"]:
            embeds = np.load(f"/workspace/{embeds_map[embeddings_source]}/{datatype}_embeddings.npy")
            ids = np.load(f"/workspace/{embeds_map[embeddings_source]}/{datatype}_ids.npy")
        
        if embeddings_source == "T5":
            embeds = np.load(f"/workspace/{embeds_map[embeddings_source]}/{datatype}_embeds.npy")
            ids = np.load(f"/workspace/{embeds_map[embeddings_source]}/{datatype}_ids.npy")
            
        embeds_list = [embeds[l, :] for l in range(embeds.shape[0])]
        self.df = pd.DataFrame({"EntryID": ids, "embed": embeds_list})
        
        if datatype == "train":
            np_labels = np.load(f"/workspace/train_targets_top{Config.num_labels}.npy")
            df_labels = pd.DataFrame(self.df['EntryID'])
            df_labels['labels_vect'] = list(np_labels)
            self.df = self.df.merge(df_labels, on="EntryID")
            
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        embed = torch.tensor(self.df.iloc[index]["embed"], dtype=torch.float32)
        if self.datatype == "train":
            targets = torch.tensor(self.df.iloc[index]["labels_vect"], dtype=torch.float32)
            return embed, targets
        if self.datatype == "test":
            protein_id = self.df.iloc[index]["EntryID"]
            return embed, protein_id

In [4]:
class MultiLayerPerceptron(nn.Module):
    """Multi-layer Perceptron model."""
    
    def __init__(self, input_dim, num_classes):
        super(MultiLayerPerceptron, self).__init__()
        self.linear1 = nn.Linear(input_dim, 864)
        self.activation1 = nn.ReLU()
        self.linear2 = nn.Linear(864, 712)
        self.activation2 = nn.ReLU()
        self.linear3 = nn.Linear(712, num_classes)
    
    def forward(self, x):
        x = self.linear1(x)
        x = self.activation1(x)
        x = self.linear2(x)
        x = self.activation2(x)
        x = self.linear3(x)
        return x


class CNN1D(nn.Module):
    """1D Convolutional Neural Network model."""
    
    def __init__(self, input_dim, num_classes):
        super(CNN1D, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=3, kernel_size=3, dilation=1, padding=1, stride=1)
        self.pool1 = nn.MaxPool1d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv1d(in_channels=3, out_channels=8, kernel_size=3, dilation=1, padding=1, stride=1)
        self.pool2 = nn.MaxPool1d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(int(8 * input_dim / 4), 864)
        self.fc2 = nn.Linear(864, num_classes)
    
    def forward(self, x):
        x = x.reshape(x.shape[0], 1, x.shape[1])
        x = self.pool1(nn.functional.tanh(self.conv1(x)))
        x = self.pool2(nn.functional.tanh(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = nn.functional.tanh(self.fc1(x))
        x = self.fc2(x)
        return x

In [5]:
def train_model(embeddings_source, model_type="linear", n_folds=5):
    """Train models using Stratified K-Fold cross-validation with label count approximation."""
    train_dataset = ProteinSequenceDataset(datatype="train", embeddings_source=embeddings_source)
    
    # Extract labels for stratification
    labels = np.stack(train_dataset.df['labels_vect'].values)
    stratify_y = np.sum(labels, axis=1)  # Use sum of labels as stratification criterion for multi-label
    
    kf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)
    
    models = []
    best_scores = []
    
    for fold, (train_idx, val_idx) in enumerate(kf.split(np.arange(len(train_dataset)), stratify_y)):
        train_set = torch.utils.data.Subset(train_dataset, train_idx)
        val_set = torch.utils.data.Subset(train_dataset, val_idx)
        
        train_dataloader = DataLoader(train_set, batch_size=Config.batch_size, shuffle=True)
        val_dataloader = DataLoader(val_set, batch_size=Config.batch_size, shuffle=True)
        
        if model_type == "linear":
            model = MultiLayerPerceptron(input_dim=embeds_dim[embeddings_source], num_classes=Config.num_labels).to(Config.device)
        elif model_type == "convolutional":
            model = CNN1D(input_dim=embeds_dim[embeddings_source], num_classes=Config.num_labels).to(Config.device)
        
        optimizer = torch.optim.Adam(model.parameters(), lr=Config.lr)
        scheduler = ReduceLROnPlateau(optimizer, mode="min", factor=0.1, patience=1, threshold=0.0001)
        criterion = nn.CrossEntropyLoss()
        f1_metric = MultilabelF1Score(num_labels=Config.num_labels).to(Config.device)
        
        best_val_score = -float('inf')
        best_model = None
        best_epoch = 0
                
        for epoch in range(Config.n_epochs):
            # TRAINING
            model.train()
            losses, scores = [], []
            for embeds, targets in tqdm(train_dataloader):
                embeds, targets = embeds.to(Config.device), targets.to(Config.device)

                optimizer.zero_grad()
                preds = model(embeds)
                loss = criterion(preds, targets)
                score = f1_metric(preds, targets)
                losses.append(loss.item())
                scores.append(score.item())
                loss.backward()

                optimizer.step()

            train_loss = np.mean(losses)
            train_score = np.mean(scores)

            # VALIDATION
            model.eval()
            losses, scores = [], []
            with torch.no_grad():
                for embeds, targets in val_dataloader:
                    embeds, targets = embeds.to(Config.device), targets.to(Config.device)
                    preds = model(embeds)
                    loss = criterion(preds, targets)
                    score = f1_metric(preds, targets)
                    losses.append(loss.item())
                    scores.append(score.item())

            val_loss = np.mean(losses)
            val_score = np.mean(scores)
            
            if val_score >= best_val_score:
                best_val_score = val_score
                best_model = copy.deepcopy(model)
                best_epoch = epoch
            scheduler.step(val_loss)

            print(f"FOLD [{fold + 1}/{n_folds}] EPOCH [{epoch + 1:02d}] TRAIN LOSS [{train_loss:.6f}] TRAIN SCORE [{train_score:.6f}] VAL LOSS [{val_loss:.6f}] VAL SCORE [{val_score:.6f}]")
        
        models.append(best_model)
        best_scores.append(best_val_score)
        print(f"FOLD [{fold + 1}] BEST EPOCH [{best_epoch + 1:02d}] BEST VALIDATION SCORE [{best_val_score:.6f}]")

        print("=" * 100)
    
    print(f"BEST SCORES: {best_scores}")
    
    return models

protbert_models = train_model("ProtBERT")

100%|██████████| 890/890 [00:03<00:00, 232.05it/s]


FOLD [1] EPOCH [01] TRAIN LOSS [138.721262] TRAIN SCORE [0.092438] VAL LOSS [138.162761] VAL SCORE [0.107106]


100%|██████████| 890/890 [00:03<00:00, 241.61it/s]


FOLD [1] EPOCH [02] TRAIN LOSS [136.675289] TRAIN SCORE [0.120655] VAL LOSS [136.479465] VAL SCORE [0.127398]


100%|██████████| 890/890 [00:03<00:00, 241.47it/s]


...

FOLD [5] EPOCH [18] TRAIN LOSS [133.301143] TRAIN SCORE [0.159776] VAL LOSS [133.740755] VAL SCORE [0.156871]


100%|██████████| 890/890 [00:03<00:00, 235.39it/s]


FOLD [5] EPOCH [19] TRAIN LOSS [133.307326] TRAIN SCORE [0.159978] VAL LOSS [133.745292] VAL SCORE [0.157402]


100%|██████████| 890/890 [00:03<00:00, 232.33it/s]


FOLD [5] EPOCH [20] TRAIN LOSS [133.258022] TRAIN SCORE [0.159848] VAL LOSS [133.780795] VAL SCORE [0.157135]
FOLD [5] BEST VALIDATION SCORE [0.157402]


In [6]:
def predict(embeddings_source, models):
    """Generate predictions using ensemble of models."""
    test_dataset = ProteinSequenceDataset(datatype="test", embeddings_source=embeddings_source)
    test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False)
    
    for model in models:
        model.eval()
    
    labels = pd.read_csv(Config.train_labels_path, sep="\t")
    top_terms = labels.groupby("term")["EntryID"].count().sort_values(ascending=False)
    labels_names = top_terms[:Config.num_labels].index.values
    print("GENERATE PREDICTION FOR TEST SET...")
    
    protein_ids = np.empty(shape=(len(test_dataloader) * Config.num_labels,), dtype=object)
    go_terms = np.empty(shape=(len(test_dataloader) * Config.num_labels,), dtype=object)
    confidences = np.empty(shape=(len(test_dataloader) * Config.num_labels,), dtype=np.float32)
    
    for i, (embed, protein_id) in tqdm(enumerate(test_dataloader), total=len(test_dataloader)):
        embed = embed.to(Config.device)
        preds_list = []
        for model in models:
            with torch.no_grad():
                preds = nn.functional.sigmoid(model(embed)).squeeze().detach().cpu().numpy()
            preds_list.append(preds)
        avg_preds = np.mean(preds_list, axis=0)
        confidences[i * Config.num_labels:(i + 1) * Config.num_labels] = avg_preds
        protein_ids[i * Config.num_labels:(i + 1) * Config.num_labels] = protein_id[0]
        go_terms[i * Config.num_labels:(i + 1) * Config.num_labels] = labels_names
    
    submission_df = pd.DataFrame({"Id": protein_ids, "GO term": go_terms, "Confidence": confidences})
    print("PREDICTIONS DONE")
    return submission_df

submission_df = predict("ProtBERT", protbert_models)
submission_df.to_csv('submission.tsv', sep='\t', header=False, index=False)

GENERATE PREDICTION FOR TEST SET...


141865it [01:06, 2124.00it/s]


PREDICTIONS DONE


In [ ]:
!kaggle competitions submit -c cafa-6-protein-function-prediction -f submission.tsv -m "CAFA6 ProtBERT Embeddings Stratified 5-Fold"

 20%|███████▉                               | 397M/1.91G [00:11<00:46, 35.5MB/s]